In [1]:
!git clone https://github.com/ewatson2/EEL6812_DeepFool_Project
!cd EEL6812_DeepFool_Project

fatal: destination path 'EEL6812_DeepFool_Project' already exists and is not an empty directory.


In [67]:
!pip install numpy torch torchvision matplotlib pandas

In [68]:
import numpy as np
import json, torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models
from torchvision.transforms import transforms

In [69]:
from models.project_models import FC_500_150, LeNet_CIFAR, LeNet_MNIST, Net
from utils.project_utils import get_clip_bounds, evaluate_attack, display_attack

In [70]:
np.random.seed(4)
torch.manual_seed(4)

use_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()  # cuda на моем устройстве (Ubuntu WSL) не поддерживается на данный момент 

False

In [71]:
mnist_mean, mnist_std, mnist_dim = 0.5, 0.5, 28

mnist_min, mnist_max = get_clip_bounds(mnist_mean, mnist_std, mnist_dim)

mnist_min = mnist_min.to(use_device)
mnist_max = mnist_max.to(use_device)

In [72]:
mnist_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mnist_mean,
        std=mnist_std)])

mnist_tf_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=mnist_mean,
        std=mnist_std)])

mnist_tf_inv = transforms.Compose([
    transforms.Normalize(
        mean=0.0,
        std=np.divide(1.0, mnist_std)),
    transforms.Normalize(
        mean=np.multiply(-1.0, mnist_std),
        std=1.0)])

mnist_temp = datasets.MNIST(root='datasets/mnist', train=True, download=True, transform=mnist_tf_train)
mnist_train, mnist_val = random_split(mnist_temp, [50000, 10000])

mnist_test = datasets.MNIST(root='datasets/mnist', train=False, download=True, transform=mnist_tf)

In [73]:
cifar_mean = [0.491, 0.482, 0.447]
cifar_std = [0.202, 0.199, 0.201]
cifar_dim = 32

cifar_min, cifar_max = get_clip_bounds(cifar_mean, cifar_std, cifar_dim)
cifar_min = cifar_min.to(use_device)
cifar_max = cifar_max.to(use_device)

cifar_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=cifar_mean,
        std=cifar_std)])

cifar_tf_train = transforms.Compose([
    transforms.RandomCrop(
        size=cifar_dim,
        padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=cifar_mean,
        std=cifar_std)])

cifar_tf_inv = transforms.Compose([
    transforms.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=np.divide(1.0, cifar_std)),
    transforms.Normalize(
        mean=np.multiply(-1.0, cifar_mean),
        std=[1.0, 1.0, 1.0])])

cifar_temp = datasets.CIFAR10(root='datasets/cifar-10', train=True, download=True, transform=cifar_tf_train)
cifar_train, cifar_val = random_split(cifar_temp, [40000, 10000])

cifar_test = datasets.CIFAR10(root='datasets/cifar-10', train=False, download=True, transform=cifar_tf)

Files already downloaded and verified
Files already downloaded and verified


In [74]:
# настройки
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

batch_size = 64
workers = 8
fgsm_eps = 0.6

deep_args = [5, len(cifar_classes), 0.05, 50]

In [75]:
mnist_loader_train = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=workers)
mnist_loader_val = DataLoader(mnist_val, batch_size=batch_size, shuffle=False, num_workers=workers)
mnist_loader_test = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=workers)
cifar_loader_train = DataLoader(cifar_train, batch_size=batch_size, shuffle=True, num_workers=workers)
cifar_loader_val = DataLoader(cifar_val, batch_size=batch_size, shuffle=False, num_workers=workers)
cifar_loader_test = DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers=workers)

In [76]:
model = LeNet_MNIST().to(use_device)

model.load_state_dict(torch.load('weights/clean/mnist_lenet.pth', map_location=use_device))

evaluate_attack('mnist_lenet_fgsm.csv', 'results', use_device, model, mnist_loader_test, mnist_min, mnist_max, fgsm_eps, is_fgsm=True) 

print('--------------------------------------------------') 

evaluate_attack('mnist_lenet_deepfool.csv', 'results', use_device, model, mnist_loader_test, mnist_min, mnist_max, deep_args, is_fgsm=False)

# if use_device.type == 'cuda': torch.cuda.empty_cache()

FGSM Test Error : 87.89%
FGSM Robustness : 4.58e-01
FGSM Time (All Images) : 0.29 s
FGSM Time (Per Image) : 28.86 us
--------------------------------------------------
DeepFool Test Error : 98.74%
DeepFool Robustness : 9.64e-02
DeepFool Time (All Images) : 193.32 s
DeepFool Time (Per Image) : 19.33 ms


In [77]:
fgsm_eps = 0.2

model = FC_500_150().to(use_device)
model.load_state_dict(torch.load('weights/clean/mnist_fc.pth', map_location=use_device))

evaluate_attack('mnist_fc_fgsm.csv', 'results', use_device, model, mnist_loader_test, mnist_min, mnist_max, fgsm_eps, is_fgsm=True)

print('--------------------------------------------------')

evaluate_attack('mnist_fc_deepfool.csv', 'results', use_device, model, mnist_loader_test, mnist_min, mnist_max, deep_args, is_fgsm=False)

FGSM Test Error : 87.08%
FGSM Robustness : 1.56e-01
FGSM Time (All Images) : 0.15 s
FGSM Time (Per Image) : 14.99 us
--------------------------------------------------
DeepFool Test Error : 97.92%
DeepFool Robustness : 6.78e-02
DeepFool Time (All Images) : 141.81 s
DeepFool Time (Per Image) : 14.18 ms
